In [ ]:
# run this if you want to run the example from the examples folder without installing the package
import sys
sys.path.insert(0, "../")

In [ ]:
# import the package
import osmosispy
from osmosispy import utils, pytypes
from cosmos.base.v1beta1.coin_pb2 import Coin

In [ ]:
# configure
mnemonic_key = "fat patch excite gold bubble large tunnel vote fine title hover junior advice cable ordinary column mass aunt trigger lucky hope animal abandon mansion"

## Prepare the account

to start localnet clone the [osmosis-labs/osmosis](https://github.com/osmosis-labs/osmosis) repo and run the following command

```bash
make localnet-build
make localnet-start
```

In [ ]:
# authorize in the localnet
network = osmosispy.Network.localnet()
trader = osmosispy.Sdk
    .authorize(key=mnemonic_key)
    .with_network(network)
    .with_config(pytypes.TxConfig())

# check the address
print(trader.address)

## Check balances of the account

In [ ]:
from osmosis_proto.proto.cosmos.bank.v1beta1 import (
  query_pb2 as bank_query,
  query_pb2_grpc as bank_query_grpc,
)

In [ ]:
stub_bank = bank_query_grpc.QueryStub(trader.query.chain_channel)
result = utils.deserialize(
    stub_bank.AllBalances(
        bank_query.QueryAllBalancesRequest(address=trader.address)
    )
)

print(result["balances"])


## Query Pools

In [ ]:
from osmosis_proto.proto.osmosis.gamm.v1beta1 import (
  query_pb2 as gamm_query,
  query_pb2_grpc as gamm_query_grpc,
)

In [ ]:
stub_gamm = gamm_query_grpc.QueryStub(trader.query.chain_channel)
result = utils.deserialize(
    stub_gamm.Pools(
        gamm_query.QueryPoolsRequest()
    )
)

print(result["pools"])

## Create Pools

In [ ]:

from osmosis_proto.proto.osmosis.gamm.pool_models.stableswap import (
    tx_pb2 as stableswap_tx,
    tx_pb2_grpc as stableswap_tx_grpc,
)
from osmosis.gamm.pool_models.stableswap.stableswap_pool_pb2 import PoolParams

In [ ]:
pool_params = PoolParams(
    swap_fee="1.0",
    exit_fee="1.0",
)

msg = stableswap_tx.MsgCreateStableswapPool(
    sender=trader.address,
    pool_params=pool_params,
    initial_pool_liquidity=[Coin("atom", 100), Coin("osmo", 100)],
    scaling_factors=[1, 1],
    future_pool_governor="",
)

stub_stable_swap = stableswap_tx_grpc.MsgStub(trader.query.chain_channel)
result = utils.deserialize(
    stub_stable_swap.CreateStableswapPool(msg)
)

## Query Locks

In [ ]:
from osmosis_proto.proto.osmosis.lockup import (
    query_pb2 as lockup_query,
    query_pb2_grpc as lockup_query_grpc,
)

In [ ]:
# Get account locks

stub_lockup = lockup_query_grpc.QueryStub(trader.query.chain_channel)
result = utils.deserialize(
    stub_lockup.AccountLockedDuration(
        # pass owner address string to query
        lockup_query.AccountLockedDurationRequest(owner=trader.address)
    )
)

print(result["locks"])

In [ ]:
# Lock by ID

stub_lockup = lockup_query_grpc.QueryStub(trader.query.chain_channel)
result = utils.deserialize(
    stub_lockup.LockedByID(
        # pass owner address string to query
        lockup_query.LockedRequest(lock_id=123)
    )
)

print(result)